# Using CMR to View Cloud-Hosted Datasets
### Author: Chris Battisto
### Date Authored: 1-31-22
### Date Updated: 1-30-23 TEST

### Timing

Exercise: 15 minutes

<p></p>

<div style="background:#fc9090;border:1px solid #cccccc;padding:5px 10px;"><big><b>Note:  </b>This notebook <em><strong>will only run in an environment with <a href="https://disc.gsfc.nasa.gov/information/glossary?keywords=%22earthdata%20cloud%22&amp;title=AWS%20region">us-west-2 AWS access</a></strong></em>.</big></div>

### Overview

This notebook demonstrates how to access cloud-hosted GES DISC granules using the [Commmon Metadata Repository (CMR) API](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html).

### Prerequisites

This notebook was written using Python 3.8, and requires these libraries and files: 
- xarray
- S3FS
- netrc file with valid Earthdata Login credentials.
- Approval to access the GES DISC archives with your Earthdata credentials (https://disc.gsfc.nasa.gov/earthdata-login)


### Import Libraries

In [1]:
import requests
import xarray as xr
import s3fs
import pprint

### Create a Function for CMR Catalog Requests

In [2]:
def cmr_request(params):
    response = requests.get(url,
                        params=params,
                        headers={
                            'Accept': 'application/json',
                        }
                       )
    return response

### Search CMR Catalogs and Obtain Data URLs

First, check that the CMR catalog can be accessed:

In [3]:
url = 'https://cmr.earthdata.nasa.gov/search/collections'

# Create our request for finding cloud-hosted granules, and check that we can access CMR
response = cmr_request({
                        'cloud_hosted': 'True',
                        'has_granules': 'True'
                        })

if response.status_code == 200:
    print(str(response.status_code) + ", CMR is accessible")
else:
    print(str(response.status_code) + ", CMR is not accessible, check for outages")

200, CMR is accessible


Lets see how many cloud-hosted data collections are currently in the GES DISC CMR catalog:

In [4]:
provider = 'GES_DISC'
response = cmr_request({
                        'cloud_hosted': 'True',
                        'has_granules': 'True',
                        'provider': provider, # Only look for data hosted by GES-DISC
                        })

# See how many hits are returned
hits = int(response.headers['cmr-hits'])
print(hits)

10


Here are the current GES DISC datasets available in the cloud as of March 2022:

In [5]:
for hit in range(0, hits):
    print(response.json()['feed']['entry'][hit]['dataset_id'])

MERRA-2 tavg1_2d_slv_Nx: 2d,1-Hourly,Time-Averaged,Single-Level,Assimilation,Single-Level Diagnostics 0.625 x 0.5 degree V5.12.4 (M2T1NXSLV) at GES DISC
GPM IMERG Final Precipitation L3 Half Hourly 0.1 degree x 0.1 degree V06 (GPM_3IMERGHH) at GES DISC
Sounder SIPS: Sun Synchronous 13:30 orbit Climate Hyperspectral InfraRed Product (CHIRP): Calibrated Radiances from EOS-Aqua, S-NPP, JPSS-1/NOAA-20, V2 (SNDR13CHRP1) at GES DISC
Sounder SIPS: Sun Synchronous 13:30 orbit Climate Hyperspectral InfraRed Product (CHIRP): Calibrated Radiances from EOS-Aqua, V2 (SNDR13CHRP1AQCal) at GES DISC
Sounder SIPS: Sun Synchronous 13:30 orbit Climate Hyperspectral InfraRed Product (CHIRP): Calibrated Radiances from JPSS-1/NOAA-20, V2 (SNDR13CHRP1J1Cal) at GES DISC
Sounder SIPS: Sun Synchronous 13:30 orbit Climate Hyperspectral InfraRed Product (CHIRP): Calibrated Radiances from S-NPP, V2 (SNDR13CHRP1SNCal) at GES DISC
Sounder SIPS: AQUA AIRS IR + MW Level 2 CLIMCAPS: Atmosphere, cloud and surface geophy

Once we know which datasets are cloud hosted, we can obtain individual granule S3 URLs by querying https://cmr.earthdata.nasa.gov/search/granules. By querying a JSON response of the granule that we want, we can obtain the new OPeNDAP link and S3 links. Here, we will parse out an S3 link to the AQUA AIRS IR + MW Level 2 CLIMCAPS dataset: 

In [6]:
climcaps_dataset_id = response.json()['feed']['entry'][6]['id']

url = 'https://cmr.earthdata.nasa.gov/search/granules.json'

response = cmr_request({
                        'concept_id': climcaps_dataset_id,
                        'temporal': '2015-01-01T00:00:00.000Z',
                        })

# Pretty print the JSON response, where we can see which entry contains the S3 link:
pprint.pprint(response.json()['feed']['entry'][0]['links'])



[{'href': 'https://data.gesdisc.earthdata.nasa.gov/data/Aqua_Sounder_Level2/SNDRAQIML2CCPRET.2/2014/365/SNDR.AQUA.AIRS_IM.20141231T2359.m06.g240.L2_CLIMCAPS_RET.std.v02_39.G.210422203342.nc',
  'hreflang': 'en-US',
  'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#',
  'title': 'Download '
           'SNDR.AQUA.AIRS_IM.20141231T2359.m06.g240.L2_CLIMCAPS_RET.std.v02_39.G.210422203342.nc'},
 {'href': 's3://gesdisc-cumulus-prod-protected/Aqua_Sounder_Level2/SNDRAQIML2CCPRET.2/2014/365/SNDR.AQUA.AIRS_IM.20141231T2359.m06.g240.L2_CLIMCAPS_RET.std.v02_39.G.210422203342.nc',
  'hreflang': 'en-US',
  'rel': 'http://esipfed.org/ns/fedsearch/1.1/s3#',
  'title': 'This link provides direct download access via S3 to the granule'},
 {'href': 'https://sounder.gesdisc.eosdis.nasa.gov/opendap/Aqua_Sounder_Level2/SNDRAQIML2CCPRET.2/2014/365/SNDR.AQUA.AIRS_IM.20141231T2359.m06.g240.L2_CLIMCAPS_RET.std.v02_39.G.210422203342.nc',
  'hreflang': 'en-US',
  'rel': 'http://esipfed.org/ns/fedsearch/1.1/servic

Now, we can parse out that link, and assign it to a variable:

In [7]:
climcaps_s3_link = response.json()['feed']['entry'][0]['links'][1]['href']
print(climcaps_s3_link)

s3://gesdisc-cumulus-prod-protected/Aqua_Sounder_Level2/SNDRAQIML2CCPRET.2/2014/365/SNDR.AQUA.AIRS_IM.20141231T2359.m06.g240.L2_CLIMCAPS_RET.std.v02_39.G.210422203342.nc


### ***Alternate Link Generation Method:***

For datasets that do not have their S3 links posted, their parent links can be manually switched to S3 using Python's <code>replace</code> function (for example, change <code>https://goldsmr4.gesdisc.eosdis.nasa.gov/data/MERRA2/</code> to <code>s3://gesdisc-cumulus-prod-protected/MERRA2/</code>). Remember that datasets like GPM IMERG may have different file organization structures, and it is recommended to use the GES DISC subsetting tool, CMR, or Earthdata Search to generate links.


In [8]:
# Paste link generated by GES DISC subsetter

#merra_opendap_link = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/2013/05/MERRA2_400.tavg1_2d_slv_Nx.20130531.nc4'
#print('OPeNDAP Link:', merra_opendap_link)

# Manually replace the on-prem server link with S3 for file list generation
#merra_s3_link = merra_opendap_link.replace('https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/', 
#                                 's3://gesdisc-cumulus-prod-protected/')

#print('S3 Link:', merra_s3_link)


OPeNDAP Link: https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/2013/05/MERRA2_400.tavg1_2d_slv_Nx.20130531.nc4
S3 Link: s3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2013/05/MERRA2_400.tavg1_2d_slv_Nx.20130531.nc4


#### Now that our S3 link has been obtained, we can generate our token, mount the GES DISC S3 bucket, and open our granule.

### Obtain S3 credentials and Open Bucket Granules

Remember that the credential token requires a previously generated netrc file, and that it will only last for one hour before needing to be regenerated.

In [9]:
gesdisc_s3 = "https://data.gesdisc.earthdata.nasa.gov/s3credentials"

# Define a function for S3 access credentials

def begin_s3_direct_access(url: str=gesdisc_s3):
    response = requests.get(url).json()
    return s3fs.S3FileSystem(key=response['accessKeyId'],
                             secret=response['secretAccessKey'],
                             token=response['sessionToken'],
                             client_kwargs={'region_name':'us-west-2'})

fs = begin_s3_direct_access()

# Check that the file system is intact as an S3FileSystem object, which means that token is valid
# Common causes of rejected S3 access tokens include incorrect passwords stored in the netrc file, or a non-existent netrc file
type(fs)

s3fs.core.S3FileSystem

Finally, we can open the CLIMCAPS granule in Xarray:

In [10]:
ds_climcaps_s3 = xr.open_dataset(fs.open(climcaps_s3_link))
ds_climcaps_s3

<xarray.Dataset>
Dimensions:                    (atrack: 45, xtrack: 30, fov: 9, utc_tuple: 8,
                                fov_poly: 8, spatial: 3, attitude: 3,
                                air_pres: 100, air_pres_h2o: 66,
                                air_pres_lay: 100, surf_wnum_ir: 100,
                                surf_freq_mw: 7, cld_lay: 2, bnds_1d: 2)
Coordinates: (12/15)
    lat                        (atrack, xtrack) float32 ...
    fov_lat                    (atrack, xtrack, fov) float32 ...
    lon                        (atrack, xtrack) float32 ...
    fov_lon                    (atrack, xtrack, fov) float32 ...
    subsat_lat                 (atrack) float32 ...
    subsat_lon                 (atrack) float32 ...
    ...                         ...
    surf_ir_wnum               (atrack, xtrack, surf_wnum_ir) float32 ...
  * air_pres                   (air_pres) float32 1.606 3.838 ... 1.1e+05
  * air_pres_h2o               (air_pres_h2o) float32 5.153e+03 ... 1.1e+05
  * air_pres_lay               (air_pres_lay) float32 0.2837 2.486 ... 1.085e+05
    cld_lay_lbl                (cld_lay) object ...
  * surf_freq_mw               (surf_freq_mw) float32 2.38e+10 ... 1.833e+11
Dimensions without coordinates: atrack, xtrack, fov, utc_tuple, fov_poly,
                                spatial, attitude, surf_wnum_ir, cld_lay,
                                bnds_1d
Data variables: (12/144)
    obs_id                     (atrack, xtrack) object ...
    fov_obs_id                 (atrack, xtrack, fov) object ...
    obs_time_tai93             (atrack, xtrack) datetime64[ns] ...
    obs_time_utc               (atrack, xtrack, utc_tuple) float32 ...
    lat_geoid                  (atrack, xtrack) float32 ...
    lon_geoid                  (atrack, xtrack) float32 ...
    ...                         ...
    ir_precip_est_24hr_qc      (atrack, xtrack, fov) float32 ...
    air_pres_nsurf             (atrack, xtrack) float32 ...
    air_pres_h2o_nsurf         (atrack, xtrack) float32 ...
    air_pres_lay_nsurf         (atrack, xtrack) float32 ...
    air_pres_lay_bnds          (air_pres_lay, bnds_1d) float32 ...
    mw_surf_class              (atrack, xtrack) float32 ...
Attributes: (12/82)
    keywords:                          ATMOSPHERE > ATMOSPHERIC TEMPERATURE >...
    Conventions:                       CF-1.6, ACDD-1.3
    source:                            AIRS and AMSU-A instrument telemetry
    processing_level:                  2
    product_name_type_id:              L2_CLIMCAPS_RET
    comment:                           
    ...                                ...
    AutomaticQualityFlag:              Passed
    qa_pct_data_missing:               0.0
    qa_pct_data_geo:                   100.0
    qa_pct_data_sci_mode:              100.0
    qa_no_data:                        FALSE
    metadata_link:                     http://disc.sci.gsfc.nasa.gov/

### Additional Exercise: Compare On-prem and S3 granules:

Xarray's <code>equals()</code> function can be called to compare any two Xarray data objects, or in this case, for seeing if the on-prem and S3 granules have identical data:

In [11]:
ds_merra_on_prem = xr.open_dataset(merra_opendap_link)
ds_merra_s3 = xr.open_dataset(fs.open(merra_s3_link))

# Always use equals() for checking if Xarray datasets are identical
if ds_merra_s3.equals(ds_merra_on_prem):
    print('The on-prem and S3 datasets are equal and intact')

The on-prem and S3 datasets are equal and intact
